In [1]:
#%pip install apyori

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5955 sha256=8796436fc127ee7f2703fec875abbfc55b93af3e39d7fd361a5a53baf507602b
  Stored in directory: c:\users\kaiky\appdata\local\pip\cache\wheels\c4\1a\79\20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori
Note: you may need to restart the kernel to use updated packages.


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dataset= pd.read_csv("Market_Basket_Optimisation.csv", header=None) #header none para ele não usar a primeira linha como coluna(não tem cols)
#recriar o dataset para ter o formato necessário para o apriori
transactions=[]

for i in range(0, dataset.shape[0]):
    transactions.append([str(dataset.values[i,j]) for j in range(0, dataset.shape[1])])
    

Fazer uma lista das informações dos dados que voce quer analisar (o formato para o apriori precisa estar em listas) e garantir que esses valores estejam listados como strings

In [22]:
print(transactions[:5])

[['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil'], ['burgers', 'meatballs', 'eggs', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['chutney', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['turkey', 'avocado', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']]


para escolher o minimo support nesse exemplo, queremos ver pelo menos 3 vezes o produto aparecer por dia, 7 pois a as transações são registradas por semana e 7501 pois é o total de transações(linhas) nos dados.

ficando: 3*7/7501 = 0.003 aprox.

In [23]:
from apyori import apriori
rules= apriori(transactions= transactions, min_support= 0.003, min_confidence= 0.2, min_lift= 3, min_length=2, max_length=2)

In [25]:
results= list(rules)
results

[RelationRecord(items=frozenset({'chicken', 'light cream'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)]),
 RelationRecord(items=frozenset({'mushroom cream sauce', 'escalope'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), confidence=0.3006993006993007, lift=3.790832696715049)]),
 RelationRecord(items=frozenset({'pasta', 'escalope'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pasta'}), items_add=frozenset({'escalope'}), confidence=0.3728813559322034, lift=4.700811850163794)]),
 RelationRecord(items=frozenset({'honey', 'fromage blanc'}), support=0.003332888948140248, ordered_statistics=[OrderedStatistic(items_base=frozenset({'fromage blanc'}), items_add=frozenset({'honey'}), confidence=0

In [39]:
def inspect(results):
    lhs= [tuple(result[2][0][0])[0] for result in results]
    rhs= [tuple(result[2][0][1])[0] for result in results]
    supports= [result[1] for result in results]
    confidences= [f"{result[2][0][2]*100:.2f}"+'%' for result in results]
    lifts= [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))

df_results= pd.DataFrame(inspect(results), columns= ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

In [43]:
df_results

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,light cream,chicken,0.004533,29.06%,4.843951
1,mushroom cream sauce,escalope,0.005733,30.07%,3.790833
2,pasta,escalope,0.005866,37.29%,4.700812
3,fromage blanc,honey,0.003333,24.51%,5.164271
4,herb & pepper,ground beef,0.015998,32.35%,3.291994
5,tomato sauce,ground beef,0.005333,37.74%,3.840659
6,light cream,olive oil,0.003200,20.51%,3.114710
7,whole wheat pasta,olive oil,0.007999,27.15%,4.122410
8,pasta,shrimp,0.005066,32.20%,4.506672


In [44]:
df_results.nlargest(n=df_results.shape[0]+1, columns='Lift')

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
3,fromage blanc,honey,0.003333,24.51%,5.164271
0,light cream,chicken,0.004533,29.06%,4.843951
2,pasta,escalope,0.005866,37.29%,4.700812
8,pasta,shrimp,0.005066,32.20%,4.506672
7,whole wheat pasta,olive oil,0.007999,27.15%,4.122410
5,tomato sauce,ground beef,0.005333,37.74%,3.840659
1,mushroom cream sauce,escalope,0.005733,30.07%,3.790833
4,herb & pepper,ground beef,0.015998,32.35%,3.291994
6,light cream,olive oil,0.003200,20.51%,3.114710
